<a href="https://colab.research.google.com/github/eduardo2531/lab_1/blob/main/Semestral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instale un montón de bibliotecas
!pip3 install neat-python
!pip3 install gym-retro
!pip3 install numpy
!pip3 install cloudpickle
!pip3 install opencv-python
!pip3 install matplotlib
!pip3 install scikit-image
!pip3 install virtualenv
!pip3 install pygame


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Ahora necesitarás importar todas las bibliotecas.

In [ ]:
import numpy as np
import retro
import neat
import pickle
import cv2
import pygame 
# video
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-white')
# Procesamento de imagenes
from skimage.transform import resize
from skimage.color import rgb2gray
from IPython import display

Creamos las carpetas del openai retro, clonamos el repositorio de python usando la opcion de fork.
creamos la virtual y la activamos 


In [ ]:
mkdir baselines 

In [ ]:
python3 -m venv baselines

In [ ]:
cd baselines 

In [ ]:
source bin/activate

In [ ]:
pip freeze

In [ ]:
! git clone --recursive https://github.com/openai/baselines.git gym-retro

In [ ]:
! git clone --recursive https://github.com/dsapandora/sonic_controller_data_capture.git

In [ ]:
virtualenv -p python3 env

In [ ]:
source env/bin/activate

In [ ]:
pip3 install gym-retro

In [ ]:
import retro

def main():
    env = retro.make(game='Airstriker-Genesis')
    obs = env.reset()
    while True:
        obs, rew, done, info = env.step(env.action_space.sample())
        env.render()
        if done:
            obs = env.reset()
    env.close()


if __name__ == "__main__":
    main()

git¿Qué juegos ya se han integrado? Tenga en cuenta que esto mostrará todos los entornos definidos, incluso aquellos para los que faltan ROM.
importracion de la roms

In [ ]:
python3 -m retro.import /ROMs/directory/

In [ ]:
# Create the environment: Name of folder where rom has been imported, Name of state, where to put recordings
env = retro.make("SonicTheHedgehog3-Genesis", "Level1") 
# if you want training to create bk2 files that you can convert into mp4 vidoes using 
# this command in a termianl: python -m retro.scripts.playback_movie <name-of-some-bk2-file.bk2>, use
# the following env line instead
# env = retro.make("SonicTheHedgehog3-Genesis", "Level1", record='.') 

In [ ]:
# This is function that calculates the fitness of each genome in the neat-python Population

def eval_genomes(genomes, config):
    imgarray = []
    xpos_end = 0

# It loops through each genome in the genomes popuation (created in the next cell)
    for genome_id, genome in genomes:
        ob = env.reset()
        ac = env.action_space.sample()
#         print(ob.shape)
        iny, inx, inc = env.observation_space.shape
        inx = int(inx/8)
        iny = int(iny/8)
# this line creates the kind of network you to use. The other main option is:
# net = neat.nn.FeedForwardNetwork.create(genome, config)

        net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)
        
# initialize some variables
        current_max_fitness = 0
        fitness_current = 0
        frame = 0
        counter = 0
        xpos = 0
        xpos_max = 0
        
        done = False

        while not done:            

            frame += 1
            ob = cv2.resize(ob, (inx, iny))
            ob = cv2.cvtColor(ob, cv2.COLOR_BGR2GRAY)
            ob = np.reshape(ob, (inx,iny))
            
# The following line lets you watch while training. Commenting it out will speed things up,
# but you wont be able see while it learns. Either way, you can still use playback.py to watch 
# the 'winner.pkl' file after training has completed. 
            env.render()

            imgarray = np.ndarray.flatten(ob)
            
            nnOutput = net.activate(imgarray)
            
            ob, rew, done, info = env.step(nnOutput)

# Set the variables you're tracking for reward
# With Sonic 3, we're only track 'x'
# You can create/change variables using gym-retro-integration (https://www.youtube.com/watch?v=eijRTLh6SBw )

            xpos = info['x']       

# This gives Sonic +1 fitness for every pixel he moves to the right. this is your fitness function.
# This determines how your AI will learn and you can change it however you please!
            if xpos > xpos_max:
                fitness_current += 1
                xpos_max = xpos
            
# This section determines if the AI has finished a level. If it passes that x position, 100000 is
# added to this current genomes fitness, which triggers the network to finish. 

            if xpos == 12300:
                fitness_current += 100000
                done = True

# if you're not interested in manually setting the fitness function, you can just use += rew
           # fitness_current += rew
            
            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
                
            if done or counter == 250:
                done = True
                print("Genome: ", genome_id, ", Fitness Achieved: ", fitness_current)
                
            genome.fitness = fitness_current

In [ ]:
#Setup config
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config-feedforward')
# creat population
p = neat.Population(config)

# add reporters so you can get some nice stats
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)

# save a check point file every 10 iterations
p.add_reporter(neat.Checkpointer(10))

# if you have already trained some, and want to restore, uncomment this line and change the 'neat-checkpoint-10'
# to whichever file you want to use (they'll all be named neat-checkpoint-somenumber)
#p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-10')

# this line runs the previous eval_genomes function. Once done, the best is set to winner
winner = p.run(eval_genomes)

#saves a pickle file of the winning genome.
with open('winner.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)